# 04 — Comprehensive Practice: The Satellite Uplink 🛰️

## 🎓 Recap

We have covered the three pillars of moving from C to Modern C++:

1.  The Toolchain: Using g++, splitting code into .hpp/.cpp, and automating it with CMake.
2.  Memory Safety (RAII): Replacing malloc/free with Constructors/Destructors and std::unique_ptr.
3.  Data Handling: Replacing raw arrays with std::vector, char* with std::string, and using References (&) to avoid expensive deep copies.

Now, we build a system that uses all of these at once.


---

## 🚀 The Mission: Telemetry Processor

You are writing the software for a satellite data processing unit. 

The Workflow:
1.  Initialize a "Sensor Driver" (simulated resource). This must be owned exclusively (unique_ptr).
2.  Load a mission configuration from mission_config.json.
3.  Collect data points into a dynamic buffer (std::vector).
4.  Analyze the data. Since the data is potentially huge, you must pass it to the analyzer without copying it.

### 📋 The Requirements

1.  Structure:
    * src/main.cpp: The flow controller.
    * src/sensor.cpp / include/sensor.hpp: The class managing the simulated hardware.
    * src/utils.cpp / include/utils.hpp: Helper functions for analysis.
    * CMakeLists.txt: Build script with nlohmann/json.
    
2.  Specific C++ Features to Use:
    * std::unique_ptr<Sensor>: To own the sensor.
    * std::vector<double>: To store sensor readings.
    * const std::vector<double>&: Function signature for the analyzer (Prevent Deep Copy!).
    * for (const auto& val : data): Range-based loop.


## 🛠️ Step 1: The Configuration (mission_config.json)

Create a file named mission_config.json in your project root:

```json
{
    "mission_id": "Apollo-X",
    "sensor_interval_ms": 200,
    "readings": [10.5, 10.8, 11.2, 9.8, 10.1, 15.5, 10.0]
}
```

(Note: In a real system, we'd read hardware, but here we load 'readings' from JSON to simulate input)


## 🛠️ Step 2: The Sensor Class (RAII)

In include/sensor.hpp, define a class Sensor.
* Constructor: Takes a std::string name. Prints "[Sensor] <name> Initialized".
* Destructor: Prints "[Sensor] <name> Shutdown".
* Method: std::string get_name() const.

This ensures that if our program crashes or returns early, the sensor "shuts down" automatically.


## 🛠️ Step 3: The Analysis Utils (References)

In include/utils.hpp, declare a function:

```cpp
void analyze_telemetry(const std::vector<double>& data);
```

Crucial: Note the const ... &. 
* &: Pass by reference (memory address), NOT by value (copy).
* const: Promise not to modify the data.

In the implementation (utils.cpp), loop through the data and print any value above 12.0 as a "WARNING".


## 🛠️ Step 4: Main Logic

In main.cpp:
1.  Use FetchContent / JSON library to read the config file.
2.  Create the sensor using auto sensor = std::make_unique<Sensor>("Thermal-Cam");.
3.  Load the readings array from JSON into a std::vector<double>.
4.  Call analyze_telemetry(data).


---

## 🔎 Solution Guide

Below are the file contents. Try to write them yourself first!


In [ ]:
// --- include/sensor.hpp ---
#pragma once
#include <string>
#include <iostream>

class Sensor {
    std::string name;
public:
    Sensor(std::string n) : name(n) {
        std::cout << "[System] Sensor " << name << " online." << std::endl;
    }
    ~Sensor() {
        std::cout << "[System] Sensor " << name << " offline." << std::endl;
    }
    std::string get_name() const { return name; }
};


In [ ]:
// --- include/utils.hpp ---
#pragma once
#include <vector>

// Passing by Const Reference to avoid Deep Copy
void analyze_telemetry(const std::vector<double>& data);


In [ ]:
// --- src/utils.cpp ---
#include "utils.hpp"
#include <iostream>

void analyze_telemetry(const std::vector<double>& data) {
    std::cout << "--- Starting Analysis ---" << std::endl;
    std::cout << "Data Points: " << data.size() << std::endl;

    // Range-based for loop with const reference
    for (const auto& val : data) {
        if (val > 12.0) {
            std::cout << "[WARNING] High reading detected: " << val << std::endl;
        }
    }
    std::cout << "--- Analysis Complete ---" << std::endl;
}


In [ ]:
// --- src/main.cpp ---
#include <iostream>
#include <fstream>
#include <vector>
#include <memory> // for unique_ptr
#include <nlohmann/json.hpp>

#include "sensor.hpp"
#include "utils.hpp"

using json = nlohmann::json;

int main() {
    // 1. RAII: Create the sensor. It manages its own lifetime.
    // We use make_unique (C++14 standard, widely available)
    auto sensor = std::make_unique<Sensor>("Lidar-X1");

    // 2. Load Config
    std::ifstream f("mission_config.json");
    if (!f.is_open()) {
        std::cerr << "Failed to open config!" << std::endl;
        return 1;
    }
    json config = json::parse(f);

    std::string mission = config["mission_id"];
    std::cout << "Mission: " << mission << " initialized." << std::endl;

    // 3. Load Data into Vector
    // Modern C++ converts JSON arrays to std::vector automatically
    std::vector<double> telemetry = config["readings"];

    // 4. Pass by Reference
    analyze_telemetry(telemetry);

    return 0;
} // <--- 'sensor' is destroyed here automatically. No memory leak.
